In [10]:
import pandas as pd
import numpy as np
import geopandas as gpd
from pysal.model import mgwr
# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import matplotlib

import libpysal
import spreg
import esda

import os

In [13]:
variables = pd.read_csv("../Data/modeling_tuning_grf.csv")
variables.head()

,Geography,lon,lat,his_num_311_per_property,neighbor_his_num_per_property,snow_depth,percent_below_poverty,percent_civilian_unemployed,per_capita_income,percent_no_highschool,...,percent_singleunits,percent_multiunit,percent_mobile_homes,percent_owneroccupiedunit,percent_crowding,percent_group_quarters,median_year_properties_built,median_value_properties_built,percent_no_vehicle,311_index_per_property
0,360290030001,1.089662e+06,1.059713e+06,52.272727,47.686137,108.149595,0.214156,0.000000,25159,0.000000,...,0.495575,0.000000,0.0,0.548263,0.069498,0.000000,1920.0,54000.0,0.108108,21.969697
1,360290170001,1.084503e+06,1.066127e+06,33.155080,43.266604,87.781572,0.042770,0.049180,45450,0.076112,...,0.758483,0.000000,0.0,0.817987,0.000000,0.000000,1920.0,43000.0,0.263383,13.368984
2,360290008001,1.084924e+06,1.036775e+06,29.487179,30.048097,149.570705,0.053443,0.020221,30061,0.209581,...,0.645892,0.016997,0.0,0.722380,0.000000,0.000000,1920.0,121000.0,0.016997,7.371795
3,360290166001,1.078699e+06,1.054669e+06,31.111111,40.989296,109.606188,0.281509,0.217252,13172,0.301370,...,0.497642,0.000000,0.0,0.674825,0.139860,0.000000,1910.0,23000.0,0.548951,11.111111
4,360290027031,1.080299e+06,1.056300e+06,65.689150,44.427546,109.196372,0.811749,0.035191,8253,0.356618,...,0.277533,0.292217,0.0,0.202509,0.030466,0.029255,1900.0,23000.0,0.564516,15.835777


In [15]:
gdf = gpd.read_file("../Data/shp/merge_all.shp")
gdf.head()

,geoid20,Area,GEOID20_1,A311,A311_his,property,A311_p,A311_his_p,X,Y,geometry
0,360290001101,0.672269,360290001101,23,57,215,10.697674,26.511628,1.082646e+06,1.033333e+06,"POLYGON ((1081475.891 1033586.531, 1081476.950..."
1,360290001102,0.381606,360290001102,29,102,449,6.458797,22.717149,1.082659e+06,1.035794e+06,"POLYGON ((1081493.060 1035039.531, 1081495.213..."
2,360290001103,8.632217,360290001103,27,113,244,11.065574,46.311475,1.077492e+06,1.036705e+06,"POLYGON ((1069977.276 1038883.172, 1070877.606..."
3,360290002001,0.422595,360290002001,21,116,270,7.777778,42.962963,1.082870e+06,1.041362e+06,"POLYGON ((1080804.135 1041798.808, 1081328.056..."
4,360290002002,0.197490,360290002002,35,82,292,11.986301,28.082192,1.082900e+06,1.038932e+06,"POLYGON ((1081531.628 1038471.056, 1081535.697..."


In [16]:
gdf1 = gdf[["geoid20", "geometry"]]

In [21]:
variables["Geography"] = variables["Geography"].astype(str)
gdf1["geoid20"] = gdf1["geoid20"].astype(str)
merge = pd.merge(variables, gdf1, left_on="Geography", right_on="geoid20", how="left")
del merge["geoid20"]
print(len(merge))
merge.head()

200


C:\Users\ryanz\anaconda3\envs\blizzard\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Geography,lon,lat,his_num_311_per_property,neighbor_his_num_per_property,snow_depth,percent_below_poverty,percent_civilian_unemployed,per_capita_income,percent_no_highschool,...,percent_multiunit,percent_mobile_homes,percent_owneroccupiedunit,percent_crowding,percent_group_quarters,median_year_properties_built,median_value_properties_built,percent_no_vehicle,311_index_per_property,geometry
0,360290030001,1.089662e+06,1.059713e+06,52.272727,47.686137,108.149595,0.214156,0.000000,25159,0.000000,...,0.000000,0.0,0.548263,0.069498,0.000000,1920.0,54000.0,0.108108,21.969697,"POLYGON ((1088695.967 1060278.607, 1088738.294..."
1,360290170001,1.084503e+06,1.066127e+06,33.155080,43.266604,87.781572,0.042770,0.049180,45450,0.076112,...,0.000000,0.0,0.817987,0.000000,0.000000,1920.0,43000.0,0.263383,13.368984,"POLYGON ((1083754.091 1065630.340, 1083756.966..."
2,360290008001,1.084924e+06,1.036775e+06,29.487179,30.048097,149.570705,0.053443,0.020221,30061,0.209581,...,0.016997,0.0,0.722380,0.000000,0.000000,1920.0,121000.0,0.016997,7.371795,"POLYGON ((1083822.189 1036668.770, 1083822.847..."
3,360290166001,1.078699e+06,1.054669e+06,31.111111,40.989296,109.606188,0.281509,0.217252,13172,0.301370,...,0.000000,0.0,0.674825,0.139860,0.000000,1910.0,23000.0,0.548951,11.111111,"POLYGON ((1077622.646 1054986.688, 1077958.580..."
4,360290027031,1.080299e+06,1.056300e+06,65.689150,44.427546,109.196372,0.811749,0.035191,8253,0.356618,...,0.292217,0.0,0.202509,0.030466,0.029255,1900.0,23000.0,0.564516,15.835777,"POLYGON ((1079708.478 1058074.076, 1080141.426..."


In [22]:
w = libpysal.weights.Queen.from_dataframe(merge)
moran = esda.Moran(merge["311_index_per_property"], w)
print("moran I: " + str(moran.I))

moran I: 0.22794045430501123


C:\Users\ryanz\AppData\Local\Temp\ipykernel_22448\3865115713.py:1: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = libpysal.weights.Queen.from_dataframe(merge)
C:\Users\ryanz\anaconda3\envs\blizzard\Lib\site-packages\libpysal\weights\contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 4 disconnected components.
  W.__init__(self, neighbors, ids=ids, **kw)


In [23]:
x_1 = merge[[
         'Geography',
         'his_num_311_per_property',
         'neighbor_his_num_per_property',
         'snow_depth',
         'percent_below_poverty',
         'percent_civilian_unemployed',
         'per_capita_income',
         'percent_no_highschool',
         'percent_65older',
         'percent_17younger',
         'percent_household_disability',
         'percent_single_parent_household',
         'percent_minority',
         'percent_notwell_english',
         'percent_singleunits',
         'percent_multiunit',
         'percent_mobile_homes',
         'percent_owneroccupiedunit',
         'percent_crowding',
         'percent_group_quarters',
         'median_year_properties_built',
         'median_value_properties_built',
         'percent_no_vehicle',
         '311_index_per_property'
        ]]

x_1 = x_1.set_index("Geography")
y = x_1.pop('311_index_per_property')

# Standardization function
def standarize_data(data, stats):
    return (data - stats['mean'])/ stats['std']

x_1_stats = x_1.describe().transpose()
x_2 = standarize_data(x_1,x_1_stats)

merge["lat"] = pd.to_numeric(merge['lat']) 
merge["lon"] = pd.to_numeric(merge['lon']) 
lng_lat_coords = np.column_stack([merge["lon"], merge["lat"]])

selector = mgwr.sel_bw.Sel_BW(lng_lat_coords, 
                            np.asarray(y).reshape(-1,1), 
                            np.asarray(x_2), 
                            spherical=False)

bw = selector.search(criterion='AICc') # golden section search; AIC; number of nearest neighbor

gwr_model = mgwr.gwr.GWR(lng_lat_coords, 
                       np.asarray(y).reshape(-1,1), 
                       np.asarray(x_2), 
                       bw)

gwr_results = gwr_model.fit()

bw

198.0